### Environment setup

This project uses a reproducible Conda environment stored in  
`tipping_environment.yml`.

Create and activate it with:

```bash
conda env create -f tipping_environment.yml -n tipping-bounds
conda activate tipping-bounds


In [3]:
# mc_one_tau_summary.py  ─────────────────────────────────────────
"""
• Single cut-off τ° = 50%; coverage = band contains Δ = 4
• Hybrid uses DKW–calibrated OUTWARD EMPIRICAL QUANTILES (not additive y-padding).
• One-sided DKW for design F (χ²₃, known lower bound 0).
• r_k is fixed deterministically: r_k = ε_k if ε_k ≤ 1/4, else 1/2 − ε_k − 1/N_k.
• Prints hit-rates and average lower / upper limits of both bands.
"""

import numpy as np
import scipy.stats as st

rng = np.random.default_rng(27062025)

# ─── constants ─────────────────────────────────────────────────
B, n   = 2_000, 50
T_SET  = [1, 2, 5]
TAU    = 50
Δ      = 4
ALPHA  = 0.05
c_M    = st.norm.ppf(1-ALPHA/2)          # 1.96  (Manski)
c_H    = st.norm.ppf(1-ALPHA/4)          # 2.24  (Hybrid)
P_TR   = 0.30
N_BIG  = 2_000_000

# full support known on both sides (used by “G” only)
SUPPORT_BOTH = {'G': (-5, 5)}
# only LOWER bound known (used by “F”)
SUPPORT_LOWER = {'F': 0.0}

# ─── DGP helpers ──────────────────────────────────────────────
def y0_A(N): return rng.normal(size=N)

def y0_B(N):  # t3 scaled to unit variance
    return st.t(df=3).rvs(size=N, random_state=rng) / np.sqrt(3)

def y0_E(N):  # rare extreme point masses at ±10 with prob 0.002 each
    y = rng.normal(size=N)
    p = rng.random(N)
    y[p < 0.002]  = -10
    y[p > 0.998]  = +10
    return y

def y0_F(N): return rng.chisquare(df=3, size=N)          # χ²₃ ≥ 0
def y0_G(N): return rng.uniform(-5, 5, N)

def ar1(N, T):
    eps = rng.normal(size=(n, T))
    y = np.zeros_like(eps)
    y[:, 0] = eps[:, 0]
    for t in range(1, T):
        y[:, t] = 0.4 * y[:, t-1] + eps[:, t]
    return y.ravel()

def treat_rand(N): return rng.binomial(1, P_TR, N)

def treat_logit(y0, b):
    p = 1 / (1 + np.exp(-(b * y0 + rng.normal(scale=0.5, size=y0.size))))
    return rng.binomial(1, p)

# ─── Manski bounds given support endpoints (a,b) ──────────────
def manski_from_ab(y1, y0, a, b):
    n1, n0 = len(y1), len(y0)
    n = n1 + n0
    p1, p0 = n1 / n, n0 / n
    d1, d0 = y1.mean(), y0.mean()
    L = p1*d1 + p0*a - p1*b - p0*d0
    U = p1*d1 + p0*b - p1*a - p0*d0
    return L, U

def se_delta_manski(y1, y0, a, b):
    """Delta-method SEs for Manski bounds with fixed endpoints (a,b)."""
    n1, n0 = len(y1), len(y0)
    n = n1 + n0
    p1, p0 = n1 / n, n0 / n
    d1, d0 = y1.mean(), y0.mean()
    gL = np.array([p1, -p0, d1 - b, a - d0])
    gU = np.array([p1, -p0, d1 - a, b - d0])
    var1 = y1.var(ddof=1) / n1
    var0 = y0.var(ddof=1) / n0
    varp = p1 * p0 / n  # simple binomial approx
    Ω = np.diag([var1, var0, varp, varp])
    return np.sqrt(gL @ Ω @ gL), np.sqrt(gU @ Ω @ gU)

# ─── Hybrid endpoints: DKW empirical quantiles ────────────────
def fixed_rk(eps, Nk):
    """Deterministic choice for r_k."""
    return eps if eps <= 0.25 else (0.5 - eps - 1.0 / Nk)

def order_stat(x, idx_ceil):
    """Return the ceil-indexed order statistic (1-based), clamped to [1, len(x)]."""
    idx = int(np.clip(int(idx_ceil), 1, len(x)))
    return np.partition(x, idx - 1)[idx - 1]

def hybrid_endpoints(y1_obs, y0_obs, alpha, one_sided_lower_known=None):
    """
    Returns endpoints (L1, U1, L0, U0) for groups k=1 and k=0 using DKW-quantile endpoints.
    If one_sided_lower_known is not None (e.g., 0.0), set Lk to that constant and
    use one-sided DKW only for the upper endpoint.
    """
    n1, n0 = len(y1_obs), len(y0_obs)

    if one_sided_lower_known is None:
        # two-sided DKW radius for sup |F_n - F| ≤ ε  with prob ≥ 1-α  ⇒ ε = sqrt(log(8/α)/(2N_k))
        eps1 = np.sqrt(np.log(8 / alpha) / (2 * n1))
        eps0 = np.sqrt(np.log(8 / alpha) / (2 * n0))
    else:
        # one-sided (upper) DKW: sup(F_n - F) ≤ ε with prob ≥ 1-α  ⇒ ε = sqrt(log(4/α)/(2N_k))
        eps1 = np.sqrt(np.log(4 / alpha) / (2 * n1))
        eps0 = np.sqrt(np.log(4 / alpha) / (2 * n0))

    r1 = fixed_rk(eps1, n1)
    r0 = fixed_rk(eps0, n0)

    if one_sided_lower_known is None:
        # Lk = F_n^{-1}(r_k + ε_k), Uk = F_n^{-1}(1 - r_k - ε_k)
        L1 = order_stat(y1_obs, np.ceil((r1 + eps1) * n1))
        U1 = order_stat(y1_obs, np.ceil((1 - r1 - eps1) * n1))
        L0 = order_stat(y0_obs, np.ceil((r0 + eps0) * n0))
        U0 = order_stat(y0_obs, np.ceil((1 - r0 - eps0) * n0))
    else:
        λ = float(one_sided_lower_known)
        L1 = λ
        L0 = λ
        U1 = order_stat(y1_obs, np.ceil((1 - r1 - eps1) * n1))
        U0 = order_stat(y0_obs, np.ceil((1 - r0 - eps0) * n0))

    return L1, U1, L0, U0

def se_delta_hybrid(y1, y0, L1, U1, L0, U0):
    """Delta-method SEs for hybrid bounds with endpoints (L1,U1,L0,U0)."""
    n1, n0 = len(y1), len(y0)
    n = n1 + n0
    p1, p0 = n1 / n, n0 / n
    d1, d0 = y1.mean(), y0.mean()
    gL = np.array([p1, -p0, d1 - U0, L1 - d0])
    gU = np.array([p1, -p0, d1 - L0, U1 - d0])
    var1 = y1.var(ddof=1) / n1
    var0 = y0.var(ddof=1) / n0
    varp = p1 * p0 / n
    Ω = np.diag([var1, var0, varp, varp])
    return np.sqrt(gL @ Ω @ gL), np.sqrt(gU @ Ω @ gU)

# ─── oracle support + moments at τ° (for reporting) ───────────
def oracle(lab):
    if lab in SUPPORT_BOTH:
        a, b = SUPPORT_BOTH[lab]
        y0_big = globals()[f'y0_{lab}'](N_BIG)
        D_big  = treat_rand(N_BIG)
    elif lab in SUPPORT_LOWER:
        a = SUPPORT_LOWER[lab]      # known lower = 0
        y0_big = y0_F(N_BIG)
        D_big  = treat_rand(N_BIG)
        b = y0_big.max() + Δ        # latent treated tail
    else:                           # unknown support
        y0_big = ar1(N_BIG, 1) if lab in 'CD' else globals()[f'y0_{lab}'](N_BIG)
        D_big  = treat_rand(N_BIG) if lab in 'ABEF' else \
                 treat_logit(y0_big, -0.5 if lab == 'C' else 0.5)
        a, b = y0_big.min(), y0_big.max() + Δ
    Y_big = y0_big + Δ * D_big
    m = D_big >= TAU / 100
    d1, d0 = Y_big[m].mean(), Y_big[~m].mean()
    p1 = m.mean();  p0 = 1 - p1
    return a, b, d1, d0, p1, p0

ORACLE = {lab: oracle(lab) for lab in "ABCDEFG"}

# ─── replication ──────────────────────────────────────────────
def replicate(lab, T):
    N = n * T
    y0 = (y0_A if lab == 'A' else y0_B if lab == 'B' else
          (lambda N: ar1(N, T)) if lab in 'CD' else
          y0_E if lab == 'E' else y0_F if lab == 'F' else
          y0_G)(N)

    D = treat_rand(N) if lab in 'ABEFG' else \
        treat_logit(y0, -0.5 if lab == 'C' else 0.5)

    a_star, b_star, *_ = ORACLE[lab]
    Y = y0 + Δ * D

    # Analyst support input for Manski (unchanged logic)
    if lab in SUPPORT_BOTH:
        a_hat, b_hat = a_star, b_star
        one_sided_lower = None
    elif lab in SUPPORT_LOWER:            # only lower bound known
        a_hat = SUPPORT_LOWER[lab]        # = 0
        b_hat = y0.max()                  # unknown upper
        one_sided_lower = a_hat
    else:                                 # no bounds known
        a_hat, b_hat = y0.min(), y0.max()
        one_sided_lower = None

    m = D >= TAU / 100
    if (not m.any()) or ((~m).sum() == 0):
        return False, False, np.nan, np.nan, np.nan, np.nan

    # Observed outcomes by arm
    y1_obs, y0_obs = Y[m], Y[~m]
    # Potential-outcome versions for Manski (Y^(1), Y^(0))
    y1_pot, y0_pot = y0[m] + Δ, y0[~m]

    # Manski band with delta SEs (using fixed (a_hat,b_hat))
    Lm0, Um0 = manski_from_ab(y1_pot, y0_pot, a_hat, b_hat)
    sL_M, sU_M = se_delta_manski(y1_pot, y0_pot, a_hat, b_hat)
    Lm, Um = Lm0 - c_M * sL_M, Um0 + c_M * sU_M

    # HYBRID band: DKW quantile endpoints + delta SEs
    if lab in SUPPORT_BOTH:
        # when both bounds known, Hybrid = Manski
        Lh, Uh = Lm, Um
    else:
        L1, U1, L0, U0 = hybrid_endpoints(
            y1_obs, y0_obs, ALPHA,
            one_sided_lower_known=one_sided_lower
        )
        # Plug into Manski formula with these endpoints
        n1, n0 = len(y1_pot), len(y0_pot)
        Ntot = n1 + n0
        p1, p0 = n1 / Ntot, n0 / Ntot
        d1, d0 = y1_pot.mean(), y0_pot.mean()
        Lh0 = p1 * d1 + p0 * L1 - p1 * U0 - p0 * d0
        Uh0 = p1 * d1 + p0 * U1 - p1 * L0 - p0 * d0
        # Delta-method SEs for hybrid (depend on endpoints)
        sL_H, sU_H = se_delta_hybrid(y1_pot, y0_pot, L1, U1, L0, U0)
        Lh, Uh = Lh0 - c_H * sL_H, Uh0 + c_H * sU_H

    hit_m = (Lm <= Δ <= Um)
    hit_h = (Lh <= Δ <= Uh)
    return hit_h, hit_m, Lm, Um, Lh, Uh

# ─── design loop ──────────────────────────────────────────────
def run_design(lab, T):
    cov_h = cov_m = 0
    smLm = smUm = smLh = smUh = 0.0
    cnt = 0
    for _ in range(B):
        ok_h, ok_m, Lm, Um, Lh, Uh = replicate(lab, T)
        cov_h += bool(ok_h)
        cov_m += bool(ok_m)
        if not np.isnan(Lm):
            smLm += Lm; smUm += Um; smLh += Lh; smUh += Uh
            cnt += 1
    cnt = max(cnt, 1)
    return (100 * cov_h / B, 100 * cov_m / B,
            smLm / cnt, smUm / cnt, smLh / cnt, smUh / cnt)

# ─── results table ────────────────────────────────────────────
fp = lambda x: f"{x:6.2f}"
print(f"Δ-in-band coverage  τ = {TAU}%   (B={B}, n={n})")
print("Design | Hyb  Man |    Lm     Um |    Lh     Uh")
for lab in "ABCDEFG":
    for T in T_SET:
        ch, cm, Lm, Um, Lh, Uh = run_design(lab, T)
        print(f"{lab}_T{T:<2} | {fp(ch)} {fp(cm)} | {fp(Lm)} {fp(Um)} | {fp(Lh)} {fp(Uh)}")


Δ-in-band coverage  τ = 50%   (B=2000, n=50)
Design | Hyb  Man |    Lm     Um |    Lh     Uh
A_T1  |  99.05  71.70 |  -1.50   4.32 |   2.98   5.02
A_T2  |  99.60  79.95 |  -1.62   4.34 |   3.25   4.78
A_T5  | 100.00  91.55 |  -1.83   4.46 |   3.21   4.88
B_T1  |  99.90  74.70 |  -2.18   5.05 |   3.03   4.96
B_T2  | 100.00  89.50 |  -2.84   5.64 |   3.29   4.72
B_T5  | 100.00  99.15 |  -4.14   6.81 |   3.35   4.70
C_T1  |  92.50  97.90 |  -0.97   4.94 |   2.61   4.53
C_T2  |  86.80  99.90 |  -1.19   5.06 |   2.83   4.27
C_T5  | 100.00 100.00 |  -1.52   5.23 |   2.58   4.51
D_T1  |  99.70  99.90 |  -0.50   5.47 |   3.33   5.63
D_T2  |  97.00 100.00 |  -0.70   5.60 |   3.64   5.37
D_T5  | 100.00 100.00 |  -0.98   5.77 |   3.51   5.62
E_T1  |  98.75  77.10 |  -2.33   5.16 |   2.96   5.04
E_T2  |  99.75  86.00 |  -3.20   5.99 |   3.23   4.80
E_T5  | 100.00  97.45 |  -4.86   7.54 |   3.19   4.89
F_T1  |  99.70 100.00 |  -4.24   9.24 |  -1.85   6.12
F_T2  |  99.60 100.00 |  -4.52   9.94 |  -1

In [5]:
rows = []                     # collect for LaTeX
print(f"Δ-in-band coverage  τ = {TAU}%   (B={B}, n={n})")
print("Design | Hyb  Man |  Lm   Um |  Lh   Uh")
for lab in "ABCDEFG":
    for T in T_SET:
        ch, cm, Lm, Um, Lh, Uh = run_design(lab, T)
        rows.append((f"{lab}\\_T{T}", ch, cm, Lm, Um, Lh, Uh))
        print(f"{lab}_T{T:<2} | {fp(ch)} {fp(cm)} | "
              f"{fp(Lm)} {fp(Um)} | {fp(Lh)} {fp(Uh)}")

# --------- LaTeX table ----------------------------------------
print("\n% ---------- LaTeX summary ----------")
print(r"\begin{tabular}{lcc|cc|cc}")
print(r"\toprule")
print(r"Design & Hybrid & Manski & $L_m$ & $U_m$ & $L_h$ & $U_h$ \\")
print(r"\midrule")
for r in rows:
    print(f"{r[0]} & {r[1]:.1f} & {r[2]:.1f} & "
          f"{r[3]:.2f} & {r[4]:.2f} & {r[5]:.2f} & {r[6]:.2f} \\\\")
print(r"\bottomrule")
print(r"\end{tabular}")


Δ-in-band coverage  τ = 50%   (B=2000, n=50)
Design | Hyb  Man |  Lm   Um |  Lh   Uh
A_T1  |  99.35  72.70 |  -1.51   4.34 |   2.99   5.03
A_T2  |  99.65  80.85 |  -1.63   4.37 |   3.24   4.79
A_T5  | 100.00  92.20 |  -1.84   4.46 |   3.21   4.88
B_T1  |  99.85  73.30 |  -2.09   4.95 |   3.04   4.97
B_T2  | 100.00  88.35 |  -2.95   5.82 |   3.29   4.73
B_T5  | 100.00  99.05 |  -4.13   6.89 |   3.35   4.71
C_T1  |  92.20  97.95 |  -0.96   4.94 |   2.62   4.53
C_T2  |  84.40  99.85 |  -1.21   5.05 |   2.81   4.25
C_T5  |  99.85 100.00 |  -1.50   5.22 |   2.58   4.51
D_T1  |  99.90  99.80 |  -0.52   5.46 |   3.31   5.60
D_T2  |  97.15 100.00 |  -0.70   5.59 |   3.64   5.36
D_T5  | 100.00 100.00 |  -0.98   5.77 |   3.50   5.62
E_T1  |  99.25  77.65 |  -2.41   5.23 |   2.96   5.04
E_T2  |  99.55  87.70 |  -3.22   5.92 |   3.22   4.79
E_T5  | 100.00  97.25 |  -4.87   7.42 |   3.20   4.88
F_T1  |  99.90 100.00 |  -4.25   9.22 |  -1.83   6.13
F_T2  |  99.75 100.00 |  -4.55  10.00 |  -1.72   5.